In [2]:
import pandas as pd
import numpy as np

In [3]:
# read the csv "test_demo.csv" can be replace with any other filename you want to input
df = pd.read_csv("test_demo.csv")

# create a new column named "UniqueID" from 1,2,3,4,5,...to n(n is # of rows we have)
# UniqueID is using to link the Source and Associate Twitter Handle, which should create by system
# usually it is used by system only
df.insert(0, "UniqueID", range(1, len(df) + 1), True)

# showing the csv
df


,UniqueID,Source,RSS feed URLs (where available),Type,Tags,Associated Twitter Handle,Associated Publisher,Name,Text aliases
0,1,@Marianhouk,NaN,Twitter Handle,Twitter Journalists,NaN,NaN,Marian Houk,Marian Houk
1,2,http://calcalist.co.il/,NaN,News Source,Israeli|Hebrew|Financial,@calcalist,Yediot,Calcalist,Calcalist
2,3,https://www.debka.com/,NaN,News Source,Israeli|English|Hawkish|Security,@debkafile|@debka_english,Debka File,Debka File - English,Debka File


In [3]:

#### this is old input file so just ignore it####

# deal with "Other language version" first
# identify rows where "Other language version" is not null
#not_null = df.loc[pd.notna(df["Associated Publisher"]), :].index


# create a new dataframe with the values that We requested,
# based on the index rows found above.
# new_df_to_add = pd.DataFrame({'UniqueID':df.iloc[not_null,0],
#                               'Source': df.iloc[not_null,9],
#                               "RSS feed URLs (where available)":np.nan,
#                               'Type': "News Source",
#                               'Tags': df.iloc[not_null,4],
#                               'Location':df.iloc[not_null,5],
#                               'Associated Twitter Handle': np.nan,
#                               'Name':df.iloc[not_null,7],
#                               'Text aliases': np.nan,
#                               'Associated Publisher': np.nan}) 

# new_df_to_add

In [4]:
# to find every non-empty Associate Twitter Handle's rows's index and store into not_null_twitter 
# for example if row 0 exist associate twitter handle, then it will be stored in not_null_twitter 
not_null_twitter = df.loc[pd.notna(df["Associated Twitter Handle"]), :].index

# create new rows for every Associate Twitter Handle, in this case source will be the old assciated twitter handle
# Type to be "Twitter Handle", UniqueID, Tags and Name will keep same.
new_df_to_add_twitter = pd.DataFrame({'UniqueID':df.iloc[not_null_twitter,0],
                              'Source': df.iloc[not_null_twitter,5],
                              "RSS feed URLs (where available)":np.nan,
                              'Type': "Twitter Handle",
                              'Tags': df.iloc[not_null_twitter,4],
                              'Associated Twitter Handle': np.nan,
                              'Associated Publisher': np.nan,
                              'Name':df.iloc[not_null_twitter,7],        
                              'Text aliases': np.nan})

# since some of associated twitter Handle will be 2 or 3, so it use "|" to split
# and make every twitter handle as a new record
new_df_to_add_twitter["Source"] = new_df_to_add_twitter["Source"].str.split("|")
new_df_to_add_twitter = new_df_to_add_twitter.explode("Source")
new_df_to_add_twitter

,UniqueID,Source,RSS feed URLs (where available),Type,Tags,Associated Twitter Handle,Associated Publisher,Name,Text aliases
1,2,@calcalist,NaN,Twitter Handle,Israeli|Hebrew|Financial,NaN,NaN,Calcalist,NaN
2,3,@debkafile,NaN,Twitter Handle,Israeli|English|Hawkish|Security,NaN,NaN,Debka File - English,NaN
2,3,@debka_english,NaN,Twitter Handle,Israeli|English|Hawkish|Security,NaN,NaN,Debka File - English,NaN


In [5]:
# add the new rows into the origin dataframe and sorted by UniqueID
df = df.append(new_df_to_add_twitter)
df = df.sort_values(by=['UniqueID'])
df

,UniqueID,Source,RSS feed URLs (where available),Type,Tags,Associated Twitter Handle,Associated Publisher,Name,Text aliases
0,1,@Marianhouk,NaN,Twitter Handle,Twitter Journalists,NaN,NaN,Marian Houk,Marian Houk
1,2,http://calcalist.co.il/,NaN,News Source,Israeli|Hebrew|Financial,@calcalist,Yediot,Calcalist,Calcalist
1,2,@calcalist,NaN,Twitter Handle,Israeli|Hebrew|Financial,NaN,NaN,Calcalist,NaN
2,3,https://www.debka.com/,NaN,News Source,Israeli|English|Hawkish|Security,@debkafile|@debka_english,Debka File,Debka File - English,Debka File
2,3,@debkafile,NaN,Twitter Handle,Israeli|English|Hawkish|Security,NaN,NaN,Debka File - English,NaN
2,3,@debka_english,NaN,Twitter Handle,Israeli|English|Hawkish|Security,NaN,NaN,Debka File - English,NaN


In [10]:
l = [x for _,x in df.groupby('Type')]
l[0]

,UniqueID,Source,RSS feed URLs (where available),Type,Tags,Associated Twitter Handle,Associated Publisher,Name,Text aliases
1,2,http://calcalist.co.il/,NaN,News Source,Israeli|Hebrew|Financial,@calcalist,Yediot,Calcalist,Calcalist
2,3,https://www.debka.com/,NaN,News Source,Israeli|English|Hawkish|Security,@debkafile|@debka_english,Debka File,Debka File - English,Debka File


In [13]:
l[1]

,UniqueID,Source,RSS feed URLs (where available),Type,Tags,Associated Twitter Handle,Associated Publisher,Name,Text aliases
0,1,@Marianhouk,NaN,Twitter Handle,Twitter Journalists,NaN,NaN,Marian Houk,Marian Houk
1,2,@calcalist,NaN,Twitter Handle,Israeli|Hebrew|Financial,NaN,NaN,Calcalist,NaN
2,3,@debkafile,NaN,Twitter Handle,Israeli|English|Hawkish|Security,NaN,NaN,Debka File - English,NaN
2,3,@debka_english,NaN,Twitter Handle,Israeli|English|Hawkish|Security,NaN,NaN,Debka File - English,NaN


In [17]:
print('l1')
print(l[1].iloc[0,])
print('l2')
print(l[1].iloc[1,])
print('l3')
print(l[1].iloc[2,])
print('l4')
print(l[1].iloc[3,])



l1
UniqueID                                             1
Source                                     @Marianhouk
RSS feed URLs (where available)                    NaN
Type                                    Twitter Handle
Tags                               Twitter Journalists
Associated Twitter Handle                          NaN
Associated Publisher                               NaN
Name                                       Marian Houk
Text aliases                               Marian Houk
Name: 0, dtype: object
l2
UniqueID                                                  2
Source                                           @calcalist
RSS feed URLs (where available)                         NaN
Type                                         Twitter Handle
Tags                               Israeli|Hebrew|Financial
Associated Twitter Handle                               NaN
Associated Publisher                                    NaN
Name                                              Calcal

In [18]:
def get_twitter_csv(file1):
    file1.to_csv('twitter.csv', index=False)
    
def get_domain_csv(file0):
    file0.to_csv('domain.csv', index=False)
    
get_twitter_csv(l[1])
get_twitter_csv(l[0])